In [18]:
import numpy as np
from copy import copy
from functools import reduce
import networkplotter as plotter
import backend.create_swot_table as swot
import pandas as pd

In [19]:
def form_sw_table(SWOT, t_count):
    sw = SWOT.index
    sw_table = np.zeros((len(sw), len(sw)))
    
    for i in range(len(sw) - 1):
        for j in range(i + 1, len(sw)):
            weight = 0
            for k in range(len(SWOT.iloc[i])):
                weight = weight - min(SWOT.iloc[i][k], SWOT.iloc[j][k]) if k < t_count \
                else weight + min(SWOT.iloc[i][k], SWOT.iloc[j][k])
           
            
            sw_table[i, j] = round(weight, 5)
    
    sw_table = sw_table + sw_table.T
    print(sw_table[0, :])
    sw = pd.DataFrame({SWOT.index[i]: sw_table[:, i] for i in range(sw_table.shape[1])}, index=SWOT.index)
    return sw
            

class Node(object):
    
    
    def __init__(self, name, value):
        self.name = name
        self.value = value
        self.connections = dict()
    
    
    def set_connections(self, connections):
        for connection in connections:
            self.connections[connection[1]] = connection[0]
      
    
    def set_weight(self, connection):
        self.connections[connection[1]] = connection[0]
    
    
    def set_value(self, value):
        self.value = value
    
    
    def remove_connection(self, node):
        try:
            self.connections.pop(node)
        except KeyError:
            print("No connection detected")
    
    
    def __str__(self):
        return self.name
    
    
class Graph(object):
    
    
    def __init__(self, name):
        self.name = name
        self.nodes = []
        self.A = []
    
    def from_pandas(self, SWOT, SW, s_count):
        nodes = dict()
        
        for index in SWOT.index:
            value = 0
            row = SWOT.loc[index]
            for i in range(len(row)):
                value = value + row[i]
            nodes[index] = Node(index, value)
            
        for column in SWOT.columns:
            value = 0
            col = SWOT[column]
            for i in range(len(col)):
                value = value + col[i] if i < s_count else value - col[i]
            nodes[column] = Node(column, value)
        
        for index in SWOT.index:
            for column in SWOT.columns:
                if SWOT.loc[index][column] != 0:
                    nodes[index].set_weight([SWOT.loc[index][column], nodes[column]])
                    nodes[column].set_weight([SWOT.loc[index][column], nodes[index]])
        
        for index in SW.index:
            for column in SW.columns:
                nodes[index].set_weight([SW.loc[index][column], nodes[column]]) \
                if SW.loc[index][column] != 0 else 0
                
        self.set_nodes(list(nodes.values()))
        
             
            
    
    def form_subgraphs(self, clusters):
        clusters_keys = list(clusters.keys())
        clusters_values  = list(clusters.values())
        subgraphs = []
        
        for cluster in clusters_keys:
            value = reduce(lambda x, y: x.value + y.value, clusters[cluster]) \
            if len(clusters[cluster]) > 1 else clusters[cluster][0].value 
            
            subgraphs.append(Node(cluster,value))    
        
        for i in range(len(clusters)):
            connections = []
            for j in range(len(clusters)):
                weight = 0
                for node_left in clusters_values[i]:
                    for node_right in node_left.connections.keys():
                        additive = node_left.connections[node_right] if node_right in clusters_values[j] \
                        and not node_right in clusters_values[i] else 0
                        weight += additive
                connections.append([weight, subgraphs[j]])
           
            subgraphs[i].set_connections(connections)
        
        aggregated = Graph('aggregated')
        aggregated.set_nodes(subgraphs)
        return aggregated
    
    
    def set_nodes(self, nodes):
        self.nodes = nodes
     
    
    def add_node(self, node):
        self.nodes.append(node)
        
        
    def remove_node(self, node):
        try:
            self.nodes.remove(node)
        except ValueError:
            print("no node " + str(node) + " in the graph")
    
    
    def __str__(self):
        return self.name
    
    
    def form_connection_matrix(self):
        self.A = np.zeros((len(self.nodes), len(self.nodes)))
        for i in range(len(self.nodes)):
            
            connection_nodes = list(self.nodes[i].connections.keys())
            for j in range(len(self.nodes)):
                self.A[i, j] = self.nodes[i].connections[self.nodes[j]] \
                if self.nodes[j] in connection_nodes else 0
        
        return self.A
    
    
    def update_values(self, nodes_values):
        for node in self.nodes:
            node.set_value(nodes_values[node])
    
    
    def send_impulses(self, impulses):
        previous = np.zeros((len(self.nodes), 1))
        current = np.array([[node.value] for node in self.nodes])
        for impulse in impulses:
            new = current + self.A.T @  (current - previous) + impulse
            nodes_values = {self.nodes[i]: current[i, 0]   for i in range(len(self.nodes))}
            previous = current
            current = new
            self.update_values(nodes_values)
        
        print(current)
        
    def search_cycles(self, verbose = False):
        cycles = set()
        for node in self.nodes:
            layer = 0
            vertexes = dict()
            vertexes[layer] = {node}
            while True:
                nexts = set()
                previous_layers = reduce(lambda x, y: x.union(y), list(vertexes.values())[: layer]) if layer > 0 else []
                for vertex in vertexes[layer]:
                    if vertex not in previous_layers:
                        for next_vertex in vertex.connections.keys():
                            nexts.add(next_vertex)
                
                layer += 1
                if nexts != set():
                    vertexes[layer] = nexts
                else:
                    break  
            
            for length in range(1, len(vertexes)):
                if node in vertexes[length]:
                    chains = np.array([[0.,node]])
                    for layers in reversed(range(length)):
                        chain = copy(chains)
                        for nodes in vertexes[layers]:
                            for i in range(len(chain[:,-1])):
                                if chain[i][-1] in nodes.connections.keys():
                                    new_chain = np.append(chain[i], nodes)
                                    new_chain[0] = chains[i][0] + nodes.connections[chain[i][-1]]
                                    
                                    if len(chain[0]) == len(chains[0]):
                                        chains = np.hstack((chains, [[0] for k in range(chains.shape[0])]))
                                    chains = np.vstack((chains,new_chain))
                    for arr in chains:
                        temp = arr[1:][arr[1:] != 0]
                        if temp[0] == temp[-1] and len(temp) > 1:
                            cycles.add((arr[0],tuple(temp)))
                    break
        if verbose:
            for cycle in cycles:
                if cycle[0] > 0:
                    print('\nWeight = ', cycle[0])
                    print(list(map(lambda val: str(val), cycle[1])))
        return cycles
    
    
    def get_eigens(self):
        return np.linalg.eig(self.A)[0] 
    
    
    def stability(self):
        print(self.get_eigens())
        max_eigen = max(abs(self.get_eigens()))
        if max_eigen < 1:
            return True
        else:
            return False
    

In [20]:
A = Node('A', 1)
C = Node('C', 2)
B = Node('B', 3)
D = Node('D', 4)

impulse = [np.array([[0], [0.1], [0.5]]) if i < 10 else np.array([[0], [0], [0]]) for i in range(50)]

A.set_connections([[0.8, C]])
B.set_connections([[0.2, B], [0.3, C], [0.111, A]])
C.set_connections([[0.2, C], [0, B], [0.3, A]])
Cognitive_Parasha = Graph('Parasha')
Cognitive_Parasha.set_nodes([A, B, C])
Cognitive_Parasha.form_subgraphs({'Bois': [A], 'Parasites': [B, C]})
Cognitive_Parasha.form_connection_matrix()
Cognitive_Parasha.search_cycles(verbose = True)
Cognitive_Parasha.get_eigens()
Cognitive_Parasha.stability()
#Cognitive_Parasha.send_impulses(impulse)


Weight =  1.1
['A', 'C', 'A']

Weight =  0.2
['B', 'B']

Weight =  0.2
['C', 'C']
[ 0.2 -0.4  0.6]


True

In [21]:
nodes = []
for i in range(22):
    nodes.append(Node(str(i), i))

for node in nodes:
    node.set_connections([[1, neighbour] if neighbour != node else [1, nodes[-1]] for neighbour in nodes[: -1]])

full_graph = Graph('nodes')
full_graph.set_nodes(nodes)
#full_graph.search_cycles(verbose = True)

In [22]:
max(abs(np.array([1,4,-5,3])))

5

In [23]:
from backend.create_swot_table import Swot
SWOT = Swot()
sswat = SWOT.swot()[0]
SW_table = form_sw_table(sswat, 10)
swot_graph = Graph('swot')
swot_graph.from_pandas(sswat, SW_table, 12)
swot_graph.form_connection_matrix()
swot_graph.stability()
swot_graph.search_cycles(verbose=False)

drawer = plotter.NetworkXPlotter(swot_graph)
drawer.plot(edge_color='Black', paper_bgcolor='White')



[ 0.    1.86  2.81  2.95  0.17  2.18  2.51  2.66  2.9   1.38  1.73  2.41
  0.46  1.98  1.69  1.    1.1   0.12  1.23 -0.03  0.9   1.4   1.97 -0.86]
[ 2.85900846e+01 -9.97287628e+00 -5.85184014e+00 -5.55717579e+00
  4.55230816e+00 -4.55814649e+00  3.21476963e+00 -3.54710953e+00
  2.60197103e+00 -2.96205209e+00 -2.73438740e+00 -2.59085626e+00
  2.21787451e+00 -2.19425020e+00  2.02360730e+00 -1.89785794e+00
  1.65365050e+00 -1.59426798e+00 -1.46342138e+00 -1.27615855e+00
 -1.25478504e+00  1.16143264e+00 -1.00552953e+00 -9.30503691e-01
  1.05201062e+00 -7.51824701e-01  8.57329269e-01  8.26946660e-01
  7.22285091e-01 -5.13333663e-01  5.91236606e-01  4.74684136e-01
 -3.92720579e-01 -3.45837883e-01  3.67755818e-01 -2.19008844e-01
  2.85491485e-01 -7.04805143e-02  1.83114657e-01  1.46953229e-01
  1.09215258e-01  5.52607865e-02  1.13410361e-02 -1.48985263e-02]


In [24]:
SWOT

In [25]:
SW_table = form_SW_table(SWOT, 10)

NameError: name 'form_SW_table' is not defined

In [ ]:
[[0]]*6